# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('ships').getOrCreate()

In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
input_data=spark.read.csv('cruise_ship_info.csv',header=True,inferSchema=True)

In [10]:
input_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [12]:
input_data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [13]:
input_data.head(2)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name=u'Quest', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [17]:
input_data.select('Cruise_line').distinct().show()

+-----------------+
|      Cruise_line|
+-----------------+
|            Costa|
|              P&O|
|           Cunard|
|Regent_Seven_Seas|
|              MSC|
|         Carnival|
|          Crystal|
|           Orient|
|         Princess|
|        Silversea|
|         Seabourn|
| Holland_American|
|         Windstar|
|           Disney|
|        Norwegian|
|          Oceania|
|          Azamara|
|        Celebrity|
|             Star|
|  Royal_Caribbean|
+-----------------+



In [18]:
input_data.select('Passengers','length','cabins','crew').describe().show()

+-------+-----------------+-----------------+------------------+-----------------+
|summary|       Passengers|           length|            cabins|             crew|
+-------+-----------------+-----------------+------------------+-----------------+
|  count|              158|              158|               158|              158|
|   mean|18.45740506329114|8.130632911392404| 8.830000000000005|7.794177215189873|
| stddev|9.677094775143416|1.793473548054825|4.4714172221480615|3.503486564627034|
|    min|             0.66|             2.79|              0.33|             0.59|
|    max|             54.0|            11.82|              27.0|             21.0|
+-------+-----------------+-----------------+------------------+-----------------+



In [23]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Without cruise line

In [26]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density'],
                          outputCol='features')

In [27]:
output=assembler.transform(input_data)

In [28]:
output.head(1)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [29]:
final_input_data=output.select('crew','features')

In [30]:
final_input_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [31]:
final_input_data.head()

Row(crew=3.55, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))

In [33]:
train_data,test_data=final_input_data.randomSplit([0.7,0.3])

In [34]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              106|
|   mean|8.039905660377357|
| stddev|3.629392313107506|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [35]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                52|
|   mean| 7.293269230769231|
| stddev|3.2069915785541205|
|    min|              0.59|
|    max|              12.2|
+-------+------------------+



In [36]:
lr=LinearRegression(labelCol='crew')

In [37]:
lr_model=lr.fit(train_data)

In [39]:
results=lr_model.evaluate(test_data)

In [40]:
results.r2

0.9536468728174637

In [41]:
results.rootMeanSquaredError

0.6837868666631256

In [43]:
results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.15164427483216114|
| -0.4075078097133432|
| 0.36479006189180696|
|-0.01068972234742871|
| -0.9209611172223109|
|-0.31618762410185064|
|-0.12531981414318638|
| -0.2583972642159442|
|-0.41748018154664246|
|-0.08462683840223573|
| 0.18171750771537676|
|-0.27751082459488163|
|-0.12080910153453228|
|   0.683182962275426|
|-0.22775271157952304|
|-0.31758773980709964|
| -1.2195463365904002|
|-0.14769342249359774|
| -1.7984076481167826|
|  0.7648127723363274|
+--------------------+
only showing top 20 rows



# Including cruise line also in vector assembler 

In [44]:
input_data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [45]:
from pyspark.ml.feature import StringIndexer

In [47]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(input_data).transform(input_data)
indexed.head()


Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)

In [48]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','cruise_cat'],
                          outputCol='features')
output=assembler.transform(indexed)

In [49]:
output.show()


+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [67]:
final_input_data=output.select('crew','features')
train_data,test_data=final_input_data.randomSplit([0.7,0.3])

train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               108|
|   mean| 8.005370370370372|
| stddev|3.6088024331806663|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               50|
|   mean|7.338000000000002|
| stddev|3.252477862322502|
|    min|              0.6|
|    max|            12.38|
+-------+-----------------+



In [68]:
lr=LinearRegression(labelCol='crew')
lr_model=lr.fit(train_data)
results=lr_model.evaluate(test_data)

In [69]:
results.r2


0.961321840818223

In [70]:
results.rootMeanSquaredError

0.633228255342297